In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta3

In [2]:
#!import Plugins/CustomPlugin/CompanySearchPlugin.cs

In [3]:
using Microsoft.SemanticKernel;;

In [4]:

// using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;

In [5]:
IKernel kernel = new KernelBuilder().WithAzureChatCompletionService("GPT3Model", "Your Azure OpenAI Service Endpoint", "Your Azure OpenAI Service API Key").Build();

In [6]:
var planner = new SequentialPlanner(kernel);

In [7]:
// var planner = kernel.ImportSkill(new SequentialPlanner(kernel));

In [8]:
planner

Microsoft.SemanticKernel.Planners.SequentialPlanner

In [9]:

CompanySearchPlugin customSkill = new CompanySearchPlugin();

In [10]:
var custom_skill = kernel.ImportFunctions(customSkill, "CustomSkill");

In [11]:
var skillsDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Plugins";

In [12]:
var write_skill = kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory,"WritePlugin");
var email_skill = kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory,"EmailPlugin");

In [13]:
var plan = await planner.CreatePlanAsync("查詢廣州天氣把結果翻譯成中文後根據天氣情況生成穿衣提示，並結合天氣結果和穿衣提示寫一封郵件");

In [14]:
// plan.Variables.ToPlan().PlanString
plan.Steps

index value 0 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value text Guangzhou Outputs [ WEATHER_RESULT ] HasNextStep False NextStepIndex 0 Name WeatherSearch PluginName CustomSkill Description search weather RequestSettings <null> SkillName CustomSkill IsSemantic True 1 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input $WEATHER_RESULT Outputs [ TRANSLATED_WEATHER ] HasNextStep False NextStepIndex 0 Name Transalte PluginName WritePlugin Description 翻譯 RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.5 presence_penalty 0 frequency_penalty 0 SkillName WritePlugin IsSemantic True 2 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input $WEATHER_RESULT Outputs [ CLOTHING_TIPS ] HasNextStep False NextStepIndex 0 Name Tips PluginName WritePlugin Description 穿衣提示 RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.5 presence_penalty 0 frequency_penalty 0 SkillName WritePlugin IsSemantic True 3 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input Weather: $TRANSLATED_WEATHER, Clothing Tips: $CLOTHING_TIPS Outputs [ RESULT__EMAIL ] HasNextStep False NextStepIndex 0 Name WeatherMail PluginName EmailPlugin Description Write an email about weather RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 1024 temperature 0 top_p 0 presence_penalty 0 frequency_penalty 0 SkillName EmailPlugin IsSemantic True

In [15]:
plan.ToJson()

{"state":[],"steps":[{"state":[],"steps":[],"parameters":[{"Key":"text","Value":"Guangzhou"}],"outputs":["WEATHER_RESULT"],"next_step_index":0,"name":"WeatherSearch","plugin_name":"CustomSkill","description":"search weather"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"$WEATHER_RESULT"}],"outputs":["TRANSLATED_WEATHER"],"next_step_index":0,"name":"Transalte","plugin_name":"WritePlugin","description":"\u7FFB\u8BD1"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"$WEATHER_RESULT"}],"outputs":["CLOTHING_TIPS"],"next_step_index":0,"name":"Tips","plugin_name":"WritePlugin","description":"\u7A7F\u8863\u63D0\u793A"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"Weather: $TRANSLATED_WEATHER, Clothing Tips: $CLOTHING_TIPS"}],"outputs":["RESULT__EMAIL"],"next_step_index":0,"name":"WeatherMail","plugin_name":"EmailPlugin","description":"Write an email about weather"}],"parameters":[],"outputs":["RESULT__EMAIL"],"next_step_index":0,"name":"plan4a3b6411

In [16]:
var result = await kernel.RunAsync(plan);

In [17]:
result.GetValue<string>()

主題：關於廣州天氣和穿衣提示的出行提醒

親愛的同事，

我希望這封郵件能夠提醒您關於廣州的天氣情況和相應的穿衣建議。根據最新的天氣預報，廣州的溫度為2度，且有雨。因此，我強烈建議您在出行時穿上暖和的衣物，以確保您的舒適度和健康。

為了應對這樣的天氣，您可以選擇穿上厚外套、毛衣和長褲等保暖衣物。這些衣物可以有效地保持您的體溫，並防止您受到寒冷的侵襲。此外，我也建議您攜帶雨具，如雨傘或雨衣，以防止雨水淋溼身體。

請您隨時留意出行狀況，特別是在雨天。確保您的行程安全和順利是我們的首要任務。如果您需要任何幫助或有任何疑問，請隨時與我聯絡。

祝您旅途愉快！

最誠摯的問候，
[您的名字]